# Intro
I wanted to thrive in yahoo's fantasy basketball's 9-cat game.  
In the 9-cat game, I'm a "drafter" drafting basketball "players".  
As a drafter, we draft 13 players in total in a snake-draft with the other 11 drafters (12 drafters in total).   
We compete against other drafter's selected team.  
The 9 categories to compete against an opponent drafter in a 9-cat game are:  
- FG%
- FT%
- 3PTM
- Points
- Assists
- Rebounds
- Steals
- Blocks
- Turnovers
Win at least 5 categories accumlated in a whole week against an opponent drafter.   

By looking at players stats everyday costs me too much time.  
I want to design a project that can assist with my decision making in drafting players.  

This Project aims at preprocessing raw data scraped from https://www.basketball-reference.com/  
Main steps in this project: 
- Data Scraping and Preprocessing:
    - Includes EDA, filtering and joining tables using pandas.   
- Create a Team Roster Auto-Builder, based on:  
    - Weekly Player Performance Prediction.  
    - Dynamic Programming for team roster selection.  


Notes: 
Fantasy basketball drafting is actually such a complex contrained optimization problem that I would like to dig deep in another project (Multiple Knapsack Assignment Problem).  
Right now I just want to "assist" my decision making rather than creating a whole automated roster building algorithm.  



# 1. Data Scraping and Preprocessing

In [67]:
# Use basketball_reference_web_scraper package.  https://jaebradley.github.io/basketball_reference_web_scraper/  
from basketball_reference_web_scraper import client
from basketball_reference_web_scraper.data import OutputType
import pandas as pd

In [68]:
# players_season_totals
df_pst2018=pd.DataFrame.from_dict(client.players_season_totals(season_end_year=2018))
df_pst2019=pd.DataFrame.from_dict(client.players_season_totals(season_end_year=2019))
df_pst2021=pd.DataFrame.from_dict(client.players_season_totals(season_end_year=2021))
df_pst2022=pd.DataFrame.from_dict(client.players_season_totals(season_end_year=2022))
df_pst2023=pd.DataFrame.from_dict(client.players_season_totals(season_end_year=2023))
df_pst2024=pd.DataFrame.from_dict(client.players_season_totals(season_end_year=2024))

In [69]:
# view the first 5 rows of the 2018 data  
df_pst2018.head()

,slug,name,positions,age,team,games_played,games_started,minutes_played,made_field_goals,attempted_field_goals,...,made_free_throws,attempted_free_throws,offensive_rebounds,defensive_rebounds,assists,steals,blocks,turnovers,personal_fouls,points
0,abrinal01,Álex Abrines,[Position.SHOOTING_GUARD],24,Team.OKLAHOMA_CITY_THUNDER,75,8,1134,115,291,...,39,46,26,88,28,38,8,25,124,353
1,acyqu01,Quincy Acy,[Position.POWER_FORWARD],27,Team.BROOKLYN_NETS,70,8,1359,130,365,...,49,60,40,217,57,33,29,60,149,411
2,adamsst01,Steven Adams,[Position.CENTER],24,Team.OKLAHOMA_CITY_THUNDER,76,76,2487,448,712,...,160,286,384,301,88,92,78,128,215,1056
3,adebaba01,Bam Adebayo,[Position.CENTER],20,Team.MIAMI_HEAT,69,19,1368,174,340,...,129,179,118,263,101,32,41,66,138,477
4,afflaar01,Arron Afflalo,[Position.SHOOTING_GUARD],32,Team.ORLANDO_MAGIC,53,3,682,65,162,...,22,26,4,62,30,4,9,21,56,179


Data cleaning and aggregation.  
The raw data only have columns made_field_goals, attempted_field_goals.  We want the aggregated result of made_field_goals/attempted_field_goals to get the Field Goal Percentage (FGperc)  
We also want to convert the total accumulated data into daily average data.  

In [70]:
df2018 = df_pst2018
df2018 = df2018.drop(columns=['positions', 'team'])
df2018 = df2018.groupby(['slug', 'name', 'age'], as_index=False)['made_field_goals', 'attempted_field_goals', 'made_free_throws', 'attempted_free_throws', \
                         'offensive_rebounds', 'defensive_rebounds', 'games_played', 'points', 'assists', 'steals', \
                            'blocks', 'turnovers', 'made_three_point_field_goals'].sum()
df2018['FGperc'] = df2018['made_field_goals'] / df2018['attempted_field_goals']
df2018['FTperc'] = df2018['made_free_throws'] / df2018['attempted_free_throws']
df2018['rebs'] = df2018['offensive_rebounds'] + df2018['defensive_rebounds']
df2018['ppg'] = df2018['points'] / df2018['games_played']
df2018['rpg'] = df2018['rebs'] / df2018['games_played']
df2018['apg'] = df2018['assists'] / df2018['games_played']
df2018['spg'] = df2018['steals'] / df2018['games_played']
df2018['bpg'] = df2018['blocks'] / df2018['games_played']
df2018['topg'] = df2018['turnovers'] / df2018['games_played']
df2018['3pg'] = df2018['made_three_point_field_goals'] / df2018['games_played']

c:\Users\joe_c\miniconda3\envs\basketball\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


Let's have a brief understanding of what value of each categories count as good or bad.  
We use the quantiles of the data to determine three levels: elite/average/poor.  
We do it on the years 2018~2024.  
Note that data in year 2024 represents the current season that is update everyday.  

In [71]:
print("\n2018 (within top 120)")
print(f'FG%: elite: {df2018["FGperc"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2018["FGperc"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2018["FGperc"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'FT%: elite: {df2018["FTperc"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2018["FTperc"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2018["FTperc"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'3PTM: elite: {df2018["3pg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2018["3pg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2018["3pg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Points: elite: {df2018["ppg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2018["ppg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2018["ppg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Assists: elite: {df2018["apg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2018["apg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2018["apg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Rebounds: elite: {df2018["rpg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2018["rpg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2018["rpg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Steals: elite: {df2018["spg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2018["spg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2018["spg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Blocks: elite: {df2018["bpg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2018["bpg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2018["bpg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Turnovers: elite: {df2018["topg"].quantile(0.25).round(decimals=3)}, \
      avg: {df2018["topg"].quantile(0.5).round(decimals=3)}, poor: {df2018["topg"].quantile(0.75).round(decimals=3)}')

# accumulate (low total: blk, stl, turnover) -> total
# average (high total: points, ast, 3pg, reb) -> top 120


2018 (within top 120)
FG%: elite: 0.591,       avg: 0.56, poor: 0.524
FT%: elite: 0.904,       avg: 0.868, poor: 0.85
3PTM: elite: 2.318,       avg: 1.925, poor: 1.648
Points: elite: 19.427,       avg: 15.937, poor: 13.874
Assists: elite: 5.229,       avg: 3.993, poor: 3.191
Rebounds: elite: 7.665,       avg: 6.305, poor: 5.356
Steals: elite: 1.462,       avg: 1.149, poor: 1.028
Blocks: elite: 1.098,       avg: 0.828, poor: 0.652
Turnovers: elite: 0.485,       avg: 0.94, poor: 1.45


In [72]:
df2019 = df_pst2019
df2019 = df2019.drop(columns=['positions', 'team'])
df2019 = df2019.groupby(['slug', 'name', 'age'], as_index=False)['made_field_goals', 'attempted_field_goals', 'made_free_throws', 'attempted_free_throws', \
                         'offensive_rebounds', 'defensive_rebounds', 'games_played', 'points', 'assists', 'steals', \
                            'blocks', 'turnovers', 'made_three_point_field_goals'].sum()
df2019['FGperc'] = df2019['made_field_goals'] / df2019['attempted_field_goals']
df2019['FTperc'] = df2019['made_free_throws'] / df2019['attempted_free_throws']
df2019['rebs'] = df2019['offensive_rebounds'] + df2019['defensive_rebounds']
df2019['ppg'] = df2019['points'] / df2019['games_played']
df2019['rpg'] = df2019['rebs'] / df2019['games_played']
df2019['apg'] = df2019['assists'] / df2019['games_played']
df2019['spg'] = df2019['steals'] / df2019['games_played']
df2019['bpg'] = df2019['blocks'] / df2019['games_played']
df2019['topg'] = df2019['turnovers'] / df2019['games_played']
df2019['3pg'] = df2019['made_three_point_field_goals'] / df2019['games_played']

print("\n2019 (within top 120)")
print(f'FG%: elite: {df2019["FGperc"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2019["FGperc"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2019["FGperc"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'FT%: elite: {df2019["FTperc"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2019["FTperc"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2019["FTperc"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'3PTM: elite: {df2019["3pg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2019["3pg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2019["3pg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Points: elite: {df2019["ppg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2019["ppg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2019["ppg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Assists: elite: {df2019["apg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2019["apg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2019["apg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Rebounds: elite: {df2019["rpg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2019["rpg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2019["rpg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Steals: elite: {df2019["spg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2019["spg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2019["spg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Blocks: elite: {df2019["bpg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2019["bpg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2019["bpg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Turnovers: elite: {df2019["topg"].quantile(0.25).round(decimals=3)}, \
      avg: {df2019["topg"].quantile(0.5).round(decimals=3)}, poor: {df2019["topg"].quantile(0.75).round(decimals=3)}')


2019 (within top 120)
FG%: elite: 0.595,       avg: 0.548, poor: 0.518
FT%: elite: 0.889,       avg: 0.864, poor: 0.841
3PTM: elite: 2.313,       avg: 1.911, poor: 1.611
Points: elite: 20.683,       avg: 16.673, poor: 14.186
Assists: elite: 5.523,       avg: 4.029, poor: 3.279
Rebounds: elite: 8.272,       avg: 6.688, poor: 5.45
Steals: elite: 1.404,       avg: 1.132, poor: 1.0
Blocks: elite: 1.11,       avg: 0.823, poor: 0.666
Turnovers: elite: 0.529,       avg: 0.856, poor: 1.452


c:\Users\joe_c\miniconda3\envs\basketball\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [73]:
df2021 = df_pst2021
df2021 = df2021.drop(columns=['positions', 'team'])
df2021 = df2021.groupby(['slug', 'name', 'age'], as_index=False)['made_field_goals', 'attempted_field_goals', 'made_free_throws', 'attempted_free_throws', \
                         'offensive_rebounds', 'defensive_rebounds', 'games_played', 'points', 'assists', 'steals', \
                            'blocks', 'turnovers', 'made_three_point_field_goals'].sum()
df2021['FGperc'] = df2021['made_field_goals'] / df2021['attempted_field_goals']
df2021['FTperc'] = df2021['made_free_throws'] / df2021['attempted_free_throws']
df2021['rebs'] = df2021['offensive_rebounds'] + df2021['defensive_rebounds']
df2021['ppg'] = df2021['points'] / df2021['games_played']
df2021['rpg'] = df2021['rebs'] / df2021['games_played']
df2021['apg'] = df2021['assists'] / df2021['games_played']
df2021['spg'] = df2021['steals'] / df2021['games_played']
df2021['bpg'] = df2021['blocks'] / df2021['games_played']
df2021['topg'] = df2021['turnovers'] / df2021['games_played']
df2021['3pg'] = df2021['made_three_point_field_goals'] / df2021['games_played']

print("\n2021 (within top 120)")
print(f'FG%: elite: {df2021["FGperc"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2021["FGperc"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2021["FGperc"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'FT%: elite: {df2021["FTperc"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2021["FTperc"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2021["FTperc"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'3PTM: elite: {df2021["3pg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2021["3pg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2021["3pg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Points: elite: {df2021["ppg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2021["ppg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2021["ppg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Assists: elite: {df2021["apg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2021["apg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2021["apg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Rebounds: elite: {df2021["rpg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2021["rpg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2021["rpg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Steals: elite: {df2021["spg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2021["spg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2021["spg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Blocks: elite: {df2021["bpg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2021["bpg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2021["bpg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Turnovers: elite: {df2021["topg"].quantile(0.25).round(decimals=3)}, \
      avg: {df2021["topg"].quantile(0.5).round(decimals=3)}, poor: {df2021["topg"].quantile(0.75).round(decimals=3)}')


2021 (within top 120)
FG%: elite: 0.618,       avg: 0.556, poor: 0.519
FT%: elite: 0.933,       avg: 0.891, poor: 0.868
3PTM: elite: 2.736,       avg: 2.216, poor: 1.931
Points: elite: 21.933,       avg: 17.595, poor: 14.91
Assists: elite: 5.82,       avg: 4.442, poor: 3.509
Rebounds: elite: 7.932,       avg: 6.659, poor: 5.727
Steals: elite: 1.282,       avg: 1.115, poor: 1.0
Blocks: elite: 1.142,       avg: 0.932, poor: 0.749
Turnovers: elite: 0.5,       avg: 0.89, poor: 1.423


c:\Users\joe_c\miniconda3\envs\basketball\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [74]:
df2022 = df_pst2022
df2022 = df2022.drop(columns=['positions', 'team'])
df2022 = df2022.groupby(['slug', 'name', 'age'], as_index=False)['made_field_goals', 'attempted_field_goals', 'made_free_throws', 'attempted_free_throws', \
                         'offensive_rebounds', 'defensive_rebounds', 'games_played', 'points', 'assists', 'steals', \
                            'blocks', 'turnovers', 'made_three_point_field_goals'].sum()
df2022['FGperc'] = df2022['made_field_goals'] / df2022['attempted_field_goals']
df2022['FTperc'] = df2022['made_free_throws'] / df2022['attempted_free_throws']
df2022['rebs'] = df2022['offensive_rebounds'] + df2022['defensive_rebounds']
df2022['ppg'] = df2022['points'] / df2022['games_played']
df2022['rpg'] = df2022['rebs'] / df2022['games_played']
df2022['apg'] = df2022['assists'] / df2022['games_played']
df2022['spg'] = df2022['steals'] / df2022['games_played']
df2022['bpg'] = df2022['blocks'] / df2022['games_played']
df2022['topg'] = df2022['turnovers'] / df2022['games_played']
df2022['3pg'] = df2022['made_three_point_field_goals'] / df2022['games_played']

print("\n2022 (within top 120)")
print(f'FG%: elite: {df2022["FGperc"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2022["FGperc"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2022["FGperc"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'FT%: elite: {df2022["FTperc"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2022["FTperc"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2022["FTperc"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'3PTM: elite: {df2022["3pg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2022["3pg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2022["3pg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Points: elite: {df2022["ppg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2022["ppg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2022["ppg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Assists: elite: {df2022["apg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2022["apg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2022["apg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Rebounds: elite: {df2022["rpg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2022["rpg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2022["rpg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Steals: elite: {df2022["spg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2022["spg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2022["spg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Blocks: elite: {df2022["bpg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2022["bpg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2022["bpg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Turnovers: elite: {df2022["topg"].quantile(0.25).round(decimals=3)}, \
      avg: {df2022["topg"].quantile(0.5).round(decimals=3)}, poor: {df2022["topg"].quantile(0.75).round(decimals=3)}')


2022 (within top 120)
FG%: elite: 0.644,       avg: 0.571, poor: 0.533
FT%: elite: 0.976,       avg: 0.888, poor: 0.868
3PTM: elite: 2.695,       avg: 2.23, poor: 1.906
Points: elite: 21.183,       avg: 17.301, poor: 15.015
Assists: elite: 5.767,       avg: 4.439, poor: 3.561
Rebounds: elite: 8.318,       avg: 6.399, poor: 5.497
Steals: elite: 1.341,       avg: 1.152, poor: 1.017
Blocks: elite: 1.066,       avg: 0.829, poor: 0.692
Turnovers: elite: 0.469,       avg: 0.809, poor: 1.297


c:\Users\joe_c\miniconda3\envs\basketball\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [75]:
df2023 = df_pst2023
df2023 = df2023.drop(columns=['positions', 'team'])
df2023 = df2023.groupby(['slug', 'name', 'age'], as_index=False)['made_field_goals', 'attempted_field_goals', 'made_free_throws', 'attempted_free_throws', \
                         'offensive_rebounds', 'defensive_rebounds', 'games_played', 'points', 'assists', 'steals', \
                            'blocks', 'turnovers', 'made_three_point_field_goals'].sum()
df2023['FGperc'] = df2023['made_field_goals'] / df2023['attempted_field_goals']
df2023['FTperc'] = df2023['made_free_throws'] / df2023['attempted_free_throws']
df2023['rebs'] = df2023['offensive_rebounds'] + df2023['defensive_rebounds']
df2023['ppg'] = df2023['points'] / df2023['games_played']
df2023['rpg'] = df2023['rebs'] / df2023['games_played']
df2023['apg'] = df2023['assists'] / df2023['games_played']
df2023['spg'] = df2023['steals'] / df2023['games_played']
df2023['bpg'] = df2023['blocks'] / df2023['games_played']
df2023['topg'] = df2023['turnovers'] / df2023['games_played']
df2023['3pg'] = df2023['made_three_point_field_goals'] / df2023['games_played']

print("\n2023 (within top 120)")
print(f'FG%: elite: {df2023["FGperc"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2023["FGperc"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2023["FGperc"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'FT%: elite: {df2023["FTperc"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2023["FTperc"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2023["FTperc"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'3PTM: elite: {df2023["3pg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2023["3pg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2023["3pg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Points: elite: {df2023["ppg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2023["ppg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2023["ppg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Assists: elite: {df2023["apg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2023["apg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2023["apg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Rebounds: elite: {df2023["rpg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2023["rpg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2023["rpg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Steals: elite: {df2023["spg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2023["spg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2023["spg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Blocks: elite: {df2023["bpg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2023["bpg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2023["bpg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Turnovers: elite: {df2023["topg"].quantile(0.25).round(decimals=3)}, \
      avg: {df2023["topg"].quantile(0.5).round(decimals=3)}, poor: {df2023["topg"].quantile(0.75).round(decimals=3)}')


2023 (within top 120)
FG%: elite: 0.638,       avg: 0.576, poor: 0.542
FT%: elite: 1.0,       avg: 0.894, poor: 0.867
3PTM: elite: 2.647,       avg: 2.109, poor: 1.91
Points: elite: 23.219,       avg: 19.475, poor: 15.352
Assists: elite: 6.102,       avg: 4.823, poor: 3.732
Rebounds: elite: 8.455,       avg: 6.426, poor: 5.462
Steals: elite: 1.318,       avg: 1.089, poor: 0.974
Blocks: elite: 1.033,       avg: 0.783, poor: 0.633
Turnovers: elite: 0.519,       avg: 0.88, poor: 1.5


c:\Users\joe_c\miniconda3\envs\basketball\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [76]:
df2024 = df_pst2024
df2024 = df2024.drop(columns=['positions', 'team'])
df2024 = df2024.groupby(['slug', 'name', 'age'], as_index=False)['made_field_goals', 'attempted_field_goals', 'made_free_throws', 'attempted_free_throws', \
                         'offensive_rebounds', 'defensive_rebounds', 'games_played', 'points', 'assists', 'steals', \
                            'blocks', 'turnovers', 'made_three_point_field_goals'].sum()
df2024['FGperc'] = df2024['made_field_goals'] / df2024['attempted_field_goals']
df2024['FTperc'] = df2024['made_free_throws'] / df2024['attempted_free_throws']
df2024['rebs'] = df2024['offensive_rebounds'] + df2024['defensive_rebounds']
df2024['ppg'] = df2024['points'] / df2024['games_played']
df2024['rpg'] = df2024['rebs'] / df2024['games_played']
df2024['apg'] = df2024['assists'] / df2024['games_played']
df2024['spg'] = df2024['steals'] / df2024['games_played']
df2024['bpg'] = df2024['blocks'] / df2024['games_played']
df2024['topg'] = df2024['turnovers'] / df2024['games_played']
df2024['3pg'] = df2024['made_three_point_field_goals'] / df2024['games_played']

print("\n2024 (within top 120)")
print(f'FG%: elite: {df2024["FGperc"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2024["FGperc"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2024["FGperc"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'FT%: elite: {df2024["FTperc"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2024["FTperc"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2024["FTperc"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'3PTM: elite: {df2024["3pg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2024["3pg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2024["3pg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Points: elite: {df2024["ppg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2024["ppg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2024["ppg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Assists: elite: {df2024["apg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2024["apg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2024["apg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Rebounds: elite: {df2024["rpg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2024["rpg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2024["rpg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Steals: elite: {df2024["spg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2024["spg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2024["spg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Blocks: elite: {df2024["bpg"].sort_values(ascending=False).head(120).quantile(0.75).round(decimals=3)}, \
      avg: {df2024["bpg"].sort_values(ascending=False).head(120).quantile(0.5).round(decimals=3)}, poor: {df2024["bpg"].sort_values(ascending=False).head(120).quantile(0.25).round(decimals=3)}')
print(f'Turnovers: elite: {df2024["topg"].quantile(0.25).round(decimals=3)}, \
      avg: {df2024["topg"].quantile(0.5).round(decimals=3)}, poor: {df2024["topg"].quantile(0.75).round(decimals=3)}')


2024 (within top 120)
FG%: elite: 0.647,       avg: 0.575, poor: 0.536
FT%: elite: 1.0,       avg: 0.893, poor: 0.871
3PTM: elite: 2.721,       avg: 2.2, poor: 1.91
Points: elite: 23.053,       avg: 18.553, poor: 14.903
Assists: elite: 5.753,       avg: 4.617, poor: 3.673
Rebounds: elite: 8.362,       avg: 6.376, poor: 5.344
Steals: elite: 1.308,       avg: 1.108, poor: 1.0
Blocks: elite: 1.119,       avg: 0.8, poor: 0.667
Turnovers: elite: 0.4,       avg: 0.774, poor: 1.371


c:\Users\joe_c\miniconda3\envs\basketball\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


Now we got a brief understanding of what the data looks like, let's start some analytics.  

# 2. Create a Team Roster Auto-Builder

We divide the process it into 2 steps:  
1. Weekly Player Performance Prediction: Average.  
2. Dynamic Programming for team roster selection.  

# 2-1. Weekly Player Performance Prediction: Average

We will first use the average values to determine the "daily" average.  
Then we will muliply by the total games played that week for the player to get the weekly projection (average).  

Get Average 9-cat stat for each player for recent seasons.  
We want the data from years 2021~2024.  
If the player's a rookie, use only the current season's data (year=2024).  

Lets first ask: which records are in df2022 and not in df2021?

In [77]:
df2022[~df2022['name'].isin(df2021['name'])].sort_values(by='points', ascending=False).head(20)


,slug,name,age,made_field_goals,attempted_field_goals,made_free_throws,attempted_free_throws,offensive_rebounds,defensive_rebounds,games_played,...,FGperc,FTperc,rebs,ppg,rpg,apg,spg,bpg,topg,3pg
558,wagnefr01,Franz Wagner,20,456,975,189,219,84,272,79,...,0.467692,0.863014,356,15.151899,4.506329,2.924051,0.860759,0.430380,1.506329,1.215190
205,greenja05,Jalen Green,19,406,954,188,236,33,193,67,...,0.425577,0.796610,226,17.268657,3.373134,2.626866,0.656716,0.268657,2.014925,2.343284
31,barnesc01,Scottie Barnes,20,459,932,158,215,195,362,74,...,0.492489,0.734884,557,15.324324,7.527027,3.459459,1.081081,0.743243,1.837838,0.783784
123,cunnica01,Cade Cunningham,20,429,1031,142,168,58,296,64,...,0.416101,0.845238,354,17.406250,5.531250,5.562500,1.218750,0.671875,3.656250,1.781250
383,mobleev01,Evan Mobley,20,422,830,167,252,143,427,69,...,0.508434,0.662698,570,14.985507,8.260870,2.521739,0.811594,1.666667,1.927536,0.333333
381,mitchda01,Davion Mitchell,23,352,843,56,85,33,133,75,...,0.417556,0.658824,166,11.480000,2.213333,4.173333,0.733333,0.320000,1.520000,1.346667
292,joneshe01,Herbert Jones,23,274,576,137,163,98,198,78,...,0.475694,0.840491,296,9.512821,3.794872,2.141026,1.666667,0.769231,1.282051,0.730769
150,duartch01,Chris Duarte,24,268,621,90,112,41,185,55,...,0.431562,0.803571,226,13.090909,4.109091,2.072727,1.018182,0.181818,1.636364,1.709091
259,hylanbo01,Bones Hyland,21,230,571,107,125,14,174,69,...,0.402802,0.856000,188,10.115942,2.724638,2.768116,0.594203,0.289855,1.246377,1.898551
496,sengual01,Alperen Şengün,19,249,525,165,232,138,255,72,...,0.474286,0.711207,393,9.611111,5.458333,2.569444,0.819444,0.944444,2.013889,0.402778


155 players in 2022 didn't appear in 2021: wagnefr01 Franz Wagner, thompkl01 Klay Thompson, ...  
They happen to be rookies in 2022 or players that were injured in the previous year (and now recovered).  

In [78]:
df2021[~df2021['name'].isin(df2022['name'])].sort_values(by='points', ascending=False).head(20)

,slug,name,age,made_field_goals,attempted_field_goals,made_free_throws,attempted_free_throws,offensive_rebounds,defensive_rebounds,games_played,...,FGperc,FTperc,rebs,ppg,rpg,apg,spg,bpg,topg,3pg
527,willizi01,Zion Williamson,20,634,1037,369,529,167,274,61,...,0.611379,0.697543,441,27.000000,7.229508,3.704918,0.934426,0.639344,2.737705,0.163934
292,leonaka01,Kawhi Leonard,29,465,908,261,295,55,282,52,...,0.512115,0.884746,337,24.846154,6.480769,5.173077,1.557692,0.403846,2.019231,1.942308
358,murraja01,Jamal Murray,23,378,792,133,153,37,157,48,...,0.477273,0.869281,194,21.208333,4.041667,4.812500,1.333333,0.270833,2.250000,2.687500
456,simmobe01,Ben Simmons,24,325,583,176,287,93,324,58,...,0.557461,0.613240,417,14.293103,7.189655,6.913793,1.603448,0.603448,2.982759,0.051724
508,walljo01,John Wall,30,293,726,158,211,17,112,40,...,0.403581,0.748815,129,20.575000,3.225000,6.875000,1.050000,0.775000,3.525000,1.975000
371,nunnke01,Kendrick Nunn,25,319,658,56,60,21,158,56,...,0.484802,0.933333,179,14.571429,3.196429,2.642857,0.928571,0.250000,1.428571,2.178571
23,bacondw01,Dwayne Bacon,25,292,726,145,176,31,193,72,...,0.402204,0.823864,224,10.944444,3.111111,1.291667,0.625000,0.069444,0.638889,0.819444
532,wisemja01,James Wiseman,19,191,368,54,86,55,171,39,...,0.519022,0.627907,226,11.487179,5.794872,0.666667,0.282051,0.923077,1.538462,0.307692
446,saricda01,Dario Šarić,26,153,342,84,99,46,144,50,...,0.447368,0.848485,190,8.740000,3.800000,1.300000,0.600000,0.080000,1.140000,0.940000
470,sumneed01,Edmond Sumner,25,148,282,68,83,18,76,53,...,0.524823,0.819277,94,7.528302,1.773585,0.924528,0.622642,0.188679,0.962264,0.660377


90 players in 2021 didn't appear in 2022: willizi01	Zion Williamson, leonaka01	Kawhi Leonard, ...  
If we want the data from year 2021~2024, find the Set Union of those 4 years's player list where we:  
    concat -> groupby -> average (weighted).   

We want a weighted average of year 2021~2024 since year 2024 should weigh more than older years like 2021.  

In [79]:
# Weighted Sum of average stats: (We can add a 'weights' attribute to each years df. )
    # (0.45*2023stat + 0.35*2022stat + 0.2*2021stat) / (0.45*2023gamesplayed + 0.35*2022gamesplayed + 0.2*2021gamesplayed)
# 'slug', 'name', (9-cat weighted sum)... 

# set weights for each year
wts = {"2024": 0.70,
       "2023": 0.15, 
       "2022": 0.10, 
       "2021": 0.05}

df2024['wts'] = wts['2024']
df2023['wts'] = wts['2023']
df2022['wts'] = wts['2022']
df2021['wts'] = wts['2021']

def weighted_average(group, values, games_played, weight):
    tmp=[]
    gp = group[games_played]
    wt = group[weight]
    for value in values:
        val = group[value]   
        if value in ['FGperc', 'FTperc']:    
            tmp.append((val * wt).sum())
        else:
            tmp.append((val * wt).sum() / (gp*wt).sum())
    return pd.Series(tmp, index=values)

values=['FGperc', 'FTperc', 'made_three_point_field_goals', 'points', 'assists', 'rebs', 'steals', 'blocks', 'turnovers']

# Although outer join is a little faster, it will miss the rows with the "same exact values". Concatenation wouldn't miss.
df_pred_avg_21_24 = pd.concat([df2021, df2022, df2023, df2024]).fillna(0)[['slug', 'name', *values, 'games_played', 'wts']] \
    .groupby(['slug', 'name'], as_index=False) \
    .apply(weighted_average, values, 'games_played', 'wts')


Let's have a look at the results of famous players.  

In [80]:
df_pred_avg_21_24[df_pred_avg_21_24['name'].isin(['Klay Thompson', 'Stephen Curry', 'Kevin Durant', 'LeBron James', 'Nikola Jokić', \
                            'Giannis Antetokounmpo', 'Tyrese Haliburton', 'De\'Aaron Fox', 'Victor Wembanyama'])]

,slug,name,FGperc,FTperc,made_three_point_field_goals,points,assists,rebs,steals,blocks,turnovers
15,antetgi01,Giannis Antetokounmpo,0.591633,0.676133,0.630631,30.907658,5.815315,11.541667,1.159910,1.121622,3.606982
175,curryst01,Stephen Curry,0.454585,0.920659,4.638398,27.454653,5.327444,4.824499,0.952886,0.365135,3.133098
215,duranke01,Kevin Durant,0.530489,0.885445,2.110482,29.423513,5.759207,6.640227,0.811615,1.169972,3.337110
247,foxde01,De'Aaron Fox,0.478675,0.729854,2.439083,26.293124,5.973462,4.121834,1.396864,0.395657,2.655006
298,halibty01,Tyrese Haliburton,0.492497,0.865417,2.961995,20.801663,10.776722,3.985748,1.349169,0.604513,2.480998
383,jamesle01,LeBron James,0.520941,0.741796,2.299261,26.591133,7.227833,7.607143,1.302956,0.690887,3.330049
402,jokicni01,Nikola Jokić,0.582930,0.820940,1.060480,25.591241,9.043796,12.034411,1.255474,0.839416,3.152242
762,thompkl01,Klay Thompson,0.408148,0.856681,3.554839,18.615484,2.303226,3.741935,0.556129,0.499355,1.707097
818,wembavi01,Victor Wembanyama,0.312648,0.557143,1.437500,19.218750,3.062500,10.187500,1.156250,3.156250,3.125000


However, these are "daily" stats rather than "weekly".  

**Weekly Projection for All Players:**  
Because we compete against a new opponent every week, we need a player's weekly performance prediction.  
Therefore, we need to take game schedule in to consideration.  
- Weekly(Monday~Sunday) Projection = Daily Prediction(use 'Average' for now) * games played   

We will create a function getWeekProjection_2324() at the end of this subsection.  

First get the 2023-2024 season schedule from bball ref.   

In [81]:
from basketball_reference_web_scraper import client
df_schedule2024 = pd.DataFrame.from_dict(client.season_schedule(season_end_year=2024))
df_schedule2024.head()

,start_time,away_team,home_team,away_team_score,home_team_score
0,2023-10-24 23:30:00+00:00,Team.LOS_ANGELES_LAKERS,Team.DENVER_NUGGETS,107.0,119.0
1,2023-10-25 02:00:00+00:00,Team.PHOENIX_SUNS,Team.GOLDEN_STATE_WARRIORS,108.0,104.0
2,2023-10-25 23:00:00+00:00,Team.HOUSTON_ROCKETS,Team.ORLANDO_MAGIC,86.0,116.0
3,2023-10-25 23:00:00+00:00,Team.BOSTON_CELTICS,Team.NEW_YORK_KNICKS,108.0,104.0
4,2023-10-25 23:00:00+00:00,Team.WASHINGTON_WIZARDS,Team.INDIANA_PACERS,120.0,143.0


Get Player List of 2024 and join with the current team name.  
Traded Players have multiple rows in df_pst2024.  
For example, James Harden was traded at the beginning of the season so he has two records: one with the 76ers, and one with the Clippers.  

We only need to know the newest team of that player.  

In [82]:
# df_pst2024 has 489, but only 483 unique names
print(df_pst2024.shape)
print(len(df_pst2024['name'].unique()))

# find the duplicate names ==> aka players that've been traded this season
df_tmp = df_pst2024[['name']].groupby(['name'], as_index=False).size()
traded_playerlist = list(df_tmp[df_tmp['size']>1]['name'])

df_pst2024[df_pst2024['name'].isin(traded_playerlist)]


(534, 22)
522


,slug,name,positions,age,team,games_played,games_started,minutes_played,made_field_goals,attempted_field_goals,...,made_free_throws,attempted_free_throws,offensive_rebounds,defensive_rebounds,assists,steals,blocks,turnovers,personal_fouls,points
0,achiupr01,Precious Achiuwa,[Position.CENTER],24,Team.TORONTO_RAPTORS,25,0,437,78,170,...,24,42,50,86,44,16,12,29,40,193
1,achiupr01,Precious Achiuwa,[Position.POWER_FORWARD],24,Team.NEW_YORK_KNICKS,5,0,60,6,17,...,0,0,10,8,2,1,3,2,5,13
13,anunoog01,OG Anunoby,[Position.SMALL_FORWARD],26,Team.TORONTO_RAPTORS,27,27,900,157,321,...,33,46,24,82,72,28,13,43,62,408
14,anunoog01,OG Anunoby,[Position.SMALL_FORWARD],26,Team.NEW_YORK_KNICKS,5,5,167,28,51,...,5,6,10,15,5,7,1,4,17,71
31,barrerj01,RJ Barrett,[Position.SHOOTING_GUARD],23,Team.NEW_YORK_KNICKS,26,26,766,164,388,...,103,124,19,92,63,12,9,47,45,474
32,barrerj01,RJ Barrett,[Position.SHOOTING_GUARD],23,Team.TORONTO_RAPTORS,6,6,198,48,81,...,20,26,6,35,21,4,1,15,21,131
36,batumni01,Nicolas Batum,[Position.POWER_FORWARD],35,Team.LOS_ANGELES_CLIPPERS,3,0,54,3,8,...,0,0,0,7,5,3,4,1,5,8
37,batumni01,Nicolas Batum,[Position.POWER_FORWARD],35,Team.PHILADELPHIA_76ERS,22,18,550,45,88,...,9,14,24,60,49,13,16,15,42,128
104,covinro01,Robert Covington,[Position.POWER_FORWARD],33,Team.LOS_ANGELES_CLIPPERS,3,3,69,3,9,...,1,2,2,6,7,6,2,0,7,9
105,covinro01,Robert Covington,[Position.SMALL_FORWARD],33,Team.PHILADELPHIA_76ERS,26,3,419,40,89,...,21,24,29,60,17,33,16,11,49,118


Conclusion from above cell: rows that appear later (larger index) are the newer teams of that traded player.  

In [83]:

playerlist2024 = df_pst2024[['slug', 'name', 'team']]

playerlist2024['idx'] = range(0,len(playerlist2024))

# only keep the row with the larger index if rows having same names and slugs
playerlist2024 = playerlist2024.loc[playerlist2024.groupby(['slug', 'name'])['idx'].idxmax()].drop(columns=['idx'])
playerlist2024


c:\Users\joe_c\miniconda3\envs\basketball\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,slug,name,team
1,achiupr01,Precious Achiuwa,Team.NEW_YORK_KNICKS
2,adebaba01,Bam Adebayo,Team.MIAMI_HEAT
3,agbajoc01,Ochai Agbaji,Team.UTAH_JAZZ
4,aldamsa01,Santi Aldama,Team.MEMPHIS_GRIZZLIES
5,alexani01,Nickeil Alexander-Walker,Team.MINNESOTA_TIMBERWOLVES
...,...,...,...
529,youngth01,Thaddeus Young,Team.TORONTO_RAPTORS
530,youngtr01,Trae Young,Team.ATLANTA_HAWKS
531,yurtsom01,Omer Yurtseven,Team.UTAH_JAZZ
532,zelleco01,Cody Zeller,Team.NEW_ORLEANS_PELICANS


Now let's add the team information to the player projection dataframe by inner-joining these two dataframes:
1. playerlist2024 (team information)
2. df_pred_avg_21_24 (player projection)

In [84]:
df_weeklyproj_avg_24 = playerlist2024.merge(df_pred_avg_21_24, on=['slug', 'name'], how='inner')
df_weeklyproj_avg_24


,slug,name,team,FGperc,FTperc,made_three_point_field_goals,points,assists,rebs,steals,blocks,turnovers
0,achiupr01,Precious Achiuwa,Team.NEW_YORK_KNICKS,0.458351,0.590238,0.498737,7.626263,1.246212,5.421717,0.536616,0.517677,1.037879
1,adebaba01,Bam Adebayo,Team.MIAMI_HEAT,0.524354,0.783249,0.024390,20.878049,3.779204,9.865212,1.179718,0.899872,2.539153
2,agbajoc01,Ochai Agbaji,Team.UTAH_JAZZ,0.375230,0.635072,1.130014,6.507607,1.033195,2.399723,0.531120,0.430152,0.712310
3,aldamsa01,Santi Aldama,Team.MEMPHIS_GRIZZLIES,0.417646,0.601087,1.318117,9.366619,1.473609,5.031384,0.589158,0.613409,0.921541
4,alexani01,Nickeil Alexander-Walker,Team.MINNESOTA_TIMBERWOLVES,0.416720,0.697632,1.358209,7.328358,2.277842,2.003444,0.800230,0.520092,1.044776
...,...,...,...,...,...,...,...,...,...,...,...,...
517,youngth01,Thaddeus Young,Team.TORONTO_RAPTORS,0.592371,0.415534,0.164557,5.618143,1.940928,3.293249,0.932489,0.221519,0.981013
518,youngtr01,Trae Young,Team.ATLANTA_HAWKS,0.432518,0.866223,2.923077,27.351648,10.484615,3.178022,1.234066,0.165934,4.203297
519,yurtsom01,Omer Yurtseven,Team.UTAH_JAZZ,0.482061,0.607295,0.118568,4.143177,0.639821,4.507830,0.214765,0.416107,0.899329
520,zelleco01,Cody Zeller,Team.NEW_ORLEANS_PELICANS,0.450095,0.640365,0.033395,3.434137,1.102041,3.480519,0.200371,0.153989,0.528757


Now we want to know how many games does each team play this week?  

Let's get the 2023-2024 NBA game schedule.  
We also want to add a 'week' column for later grouping.  

In [85]:
import datetime

# first assume every player on the team plays. Just get the weekly schedule of the player's team. 
# view schema: team, week (starting from 2023-10-24), gamesplayed (during that week) 
# can also add columns: homegames, awaygames

df_schedule2024.start_time = df_schedule2024['start_time'].dt.tz_convert('America/Los_Angeles')
df_schedule2024.start_time = df_schedule2024.start_time.dt.date

# Create 'week' column: 
# week1: 2023-10-24 ~ 10-29, week2: 10-30 ~ 11-5, week3: 11-6 ~ 11-12, ...
# R dplyr::mutate, equivalent to pandas conditionally with list comprehension, or apply lambda

seasonStartDate = df_schedule2024.start_time[0]
seasonStartDate_weekday = datetime.timedelta(seasonStartDate.weekday())
df_schedule2024['week'] = df_schedule2024['start_time'].apply(lambda x: (x - seasonStartDate + seasonStartDate_weekday).days//7 + 1)
df_schedule2024


,start_time,away_team,home_team,away_team_score,home_team_score,week
0,2023-10-24,Team.LOS_ANGELES_LAKERS,Team.DENVER_NUGGETS,107.0,119.0,1
1,2023-10-24,Team.PHOENIX_SUNS,Team.GOLDEN_STATE_WARRIORS,108.0,104.0,1
2,2023-10-25,Team.HOUSTON_ROCKETS,Team.ORLANDO_MAGIC,86.0,116.0,1
3,2023-10-25,Team.BOSTON_CELTICS,Team.NEW_YORK_KNICKS,108.0,104.0,1
4,2023-10-25,Team.WASHINGTON_WIZARDS,Team.INDIANA_PACERS,120.0,143.0,1
...,...,...,...,...,...,...
1226,2024-04-14,Team.PHOENIX_SUNS,Team.MINNESOTA_TIMBERWOLVES,NaN,NaN,25
1227,2024-04-14,Team.LOS_ANGELES_LAKERS,Team.NEW_ORLEANS_PELICANS,NaN,NaN,25
1228,2024-04-14,Team.DALLAS_MAVERICKS,Team.OKLAHOMA_CITY_THUNDER,NaN,NaN,25
1229,2024-04-14,Team.PORTLAND_TRAIL_BLAZERS,Team.SACRAMENTO_KINGS,NaN,NaN,25


Group by week and team to know how many games does each team play this week?   

In [86]:
# groupby week and team
df_gamesplayed_weekly = df_schedule2024.copy()
df_gamesplayed_weekly['away_team'] = df_gamesplayed_weekly['away_team'].apply(lambda x: x.value)
df_gamesplayed_weekly['home_team'] = df_gamesplayed_weekly['home_team'].apply(lambda x: x.value)
df_gamesplayed_weekly

df_tmp1 = df_gamesplayed_weekly[['week', 'away_team']].groupby(['week', 'away_team'], as_index=False).size()
df_tmp2 = df_gamesplayed_weekly[['week', 'home_team']].groupby(['week', 'home_team'], as_index=False).size()
df_tmp1.rename(columns={'away_team':"team"}, inplace=True)
df_tmp2.rename(columns={'home_team':"team"}, inplace=True)
df_gamesplayed_weekly = pd.concat([df_tmp1, df_tmp2]).groupby(['week', 'team'], as_index=False).sum()
# df_gamesplayed_weekly.sort_values(by=['week', 'size'], ascending = [True, False])
df_gamesplayed_weekly


,week,team,size
0,1,ATLANTA HAWKS,3
1,1,BOSTON CELTICS,2
2,1,BROOKLYN NETS,2
3,1,CHARLOTTE HORNETS,2
4,1,CHICAGO BULLS,3
...,...,...,...
745,25,SACRAMENTO KINGS,4
746,25,SAN ANTONIO SPURS,4
747,25,TORONTO RAPTORS,4
748,25,UTAH JAZZ,4


Example: De'Aaron Fox's week6 projection  

In [87]:
# De'Aaron Fox week6 projection
week = 13
print(type(playerlist2024[playerlist2024.name=='De\'Aaron Fox'].team.values[0]))
team = playerlist2024[playerlist2024.name=='De\'Aaron Fox'].team.values[0].value
gamesPlayed_week = df_gamesplayed_weekly[(df_gamesplayed_weekly['week']==week) & (df_gamesplayed_weekly['team']==team)]['size'].iloc[0]
print(df_weeklyproj_avg_24[df_weeklyproj_avg_24['name']=='De\'Aaron Fox'].iloc[:,3:5]) # Percentage doesn't need to multiply by games played. 
df_weeklyproj_avg_24[df_weeklyproj_avg_24['name']=='De\'Aaron Fox'].iloc[:,5:] * gamesPlayed_week


<enum 'Team'>
       FGperc    FTperc
140  0.478675  0.729854


,made_three_point_field_goals,points,assists,rebs,steals,blocks,turnovers
140,4.878166,52.586248,11.946924,8.243667,2.793727,0.791315,5.310012


Create a function getWeekProjection_2324()  
The input is an nba week number, and outputs are the weekly projection for all the players in that specified week.  
We have a look at the outputs sorted by made_three_point_field_goals.   

In [88]:
# func(week) ==> weekly projection of every player
def current_week():
    seasonStartDate = df_schedule2024.start_time[0]
    seasonStartDate_weekday = datetime.timedelta(seasonStartDate.weekday())
    week = (datetime.date.today() - seasonStartDate + seasonStartDate_weekday).days//7 + 1
    return week

def getWeekProjection_2324(week=current_week()):
    df_weeklyproj_avg_24 = playerlist2024.merge(df_pred_avg_21_24, on=['slug', 'name'], how='inner')
    gamesPlayed_week = df_gamesplayed_weekly[(df_gamesplayed_weekly['week']==week) & (df_gamesplayed_weekly['team']==team)]['size'].iloc[0]
    df_weeklyproj_avg_24.iloc[:,5:] = df_weeklyproj_avg_24.iloc[:,5:] * gamesPlayed_week
    return df_weeklyproj_avg_24

df_tmp = getWeekProjection_2324(current_week())
df_tmp.sort_values(by='made_three_point_field_goals', ascending=False).head(20)




,slug,name,team,FGperc,FTperc,made_three_point_field_goals,points,assists,rebs,steals,blocks,turnovers
105,curryst01,Stephen Curry,Team.GOLDEN_STATE_WARRIORS,0.454585,0.920659,18.553592,109.818610,21.309776,19.297998,3.811543,1.460542,12.532391
454,thompkl01,Klay Thompson,Team.GOLDEN_STATE_WARRIORS,0.408148,0.856681,14.219355,74.461935,9.212903,14.967742,2.224516,1.997419,6.828387
118,doncilu01,Luka Dončić,Team.DALLAS_MAVERICKS,0.483564,0.766711,13.972414,128.395402,35.144828,33.489655,5.250575,2.174713,15.857471
268,lillada01,Damian Lillard,Team.MILWAUKEE_BUCKS,0.431823,0.912781,13.589677,107.798710,28.211613,17.610323,3.870968,0.913548,11.220645
319,mitchdo01,Donovan Mitchell,Team.CLEVELAND_CAVALIERS,0.457575,0.870621,13.279584,108.832250,21.076723,19.079324,6.486346,1.570871,10.897269
150,georgpa01,Paul George,Team.LOS_ANGELES_CLIPPERS,0.458699,0.902381,13.100775,94.351421,17.281654,23.540052,6.423773,1.509044,10.382429
21,ballla01,LaMelo Ball,Team.CHARLOTTE_HORNETS,0.436352,0.850531,13.040462,88.732177,31.514451,24.238921,5.788054,1.287091,14.242775
297,mccolcj01,CJ McCollum,Team.NEW_ORLEANS_PELICANS,0.460395,0.777904,12.857909,82.750670,20.932976,17.034853,4.589812,2.461126,7.581769
471,vanvlfr01,Fred VanVleet,Team.HOUSTON_ROCKETS,0.401763,0.877274,12.828214,74.056883,31.185438,16.182025,5.442548,2.534699,7.713311
446,tatumja01,Jayson Tatum,Team.BOSTON_CELTICS,0.474043,0.827095,12.783370,112.792123,17.934354,33.684902,4.153173,2.275711,11.326039


Notes: In the 2023-2024 season, week7 is the week of In-Season Tournament that won't been scheduled beforehand. 

# 2-2. Dynamic Programming for team roster selection.  

13-roster Team Selection Mechanism under a 12-team Snake Draft.   
Create the 13-roster Team from the Weekly Projection using similar techinques as when solving the 0/1 knapsack problem.  

Find the best 13 players to win a 9-cat game regardless of the position constraint and the snake draft constraint.  

1. Brute Force: 13 players out of 483 (mylist) or 548 (fantasypros draft list). Using recursive has an O(2^m) time complexity, and C(483, 13) = 1.062392279 E+25 is too large of a number.   

2. Dynamic Programming: 0/1 Knapsack problem  
    Given N **items** where each item has some weight and **profit** associated with it and also given a bag with **capacity** W, [i.e., the bag can hold at most W weight in it]. The goal is to put the items into the bag such that is has the **max sum of profits** possible.  
    
    N: total player list (**items**)  
    W: 13-player roster team that consists of 1PG, 1SG, 1G, 1SF, 1PF, 1F, 2C, 2Util, 3BN (**capacity**)  
    profit: 9-cat stats (**profits**)  
    goal: categorical wins (**max sum of profits**)  




Let's first solve 0/1 knapsack problem.  

Assume that the 13-player team doesn't have position constraints.  
aka. every player's weight is 1, where capacity W=13.  

The difference in our case is that: **"profit" is multidimensional** since they're 9-cat stats.  
==> we can do an "operation overload" for this multidimensional-categorical profit  
    usually: profit1=8, profit2=4, profit3=2;  
        profit1 = 8 > profit2 + profit3 = 6  
    now: profit1 = [0,2,2,2,2,2,2,2,0], profit2 = [100,0,0,0,0,0,0,1,1], profit3 = [0,100,0,0,0,0,1,0,0]   
        profit1 = [0,2,2,2,2,2,2,2,2] > profit2 + profit3 = [100,100,0,0,0,0,1,1,1]   
        profit1 wins the last 7 categories over profit2 + profit3's first 2 categories. 
        list-elementwise greaterthan.   

In [89]:
# define the new greaterthan
def profit_greaterthan(A, B): 
    # A and B are lists. 
    l = []
    for (a, b) in zip(A,B):
        if a > b:
            l.append(1)
        elif a==b:
            l.append(0)
        else:
            l.append(-1)
    if sum(l) > 0:
        return True
    else:
        return False

def profit_max(A_exclude, B_include): 
    # A, B, and idx are lists. 
    if profit_greaterthan(A_exclude, B_include):
        return A_exclude
    else:
        return B_include
    
def profit_addition(A, B):
    # A and B are lists. 
    return [sum(x) for x in zip(A, B)]


# Space optimized Approach (code is inspired by Prosun Kumar Sarkar)
def knapSack(W, wt, val, n): 
    # Making the dp array each with 9 category values initialized as zero.  
    dp = [[0]*9 for i in range(W+1)]

    # Taking first i elements 
    for i in range(1, n+1): 
        # Starting from back, 
        # so that we also have data of 
        # previous computation when taking i-1 items 
        for w in range(W, 0, -1): 
            if wt[i-1] <= w: 
                # Find the max value 
                dp[w] = profit_max(dp[w], profit_addition(dp[w-wt[i-1]], val[i-1])) 
      
    # Returning the maximum value of knapsack 
    return dp[W]
  
  
profit = [[100,0,0,0,0,0,0,1,1], [0,100,0,0,0,0,1,0,0], [0,2,2,2,2,2,2,2,0]]
weight = [1, 1, 1]
W = 2
n = len(profit)
print(knapSack(W, weight, profit, n))


profit = [1, 7, 4]
weight = [3, 8, 2]
W = 6
N = 3





[100, 2, 2, 2, 2, 2, 2, 3, 1]


However, the above doesn't tell us which items were selected.  
We need to output the selected indexes.  

In [90]:
# define the new greaterthan
def profit_greaterthan(A, B): 
    # A and B are lists. 
    l = []
    for (a, b) in zip(A,B):
        if a > b:
            l.append(1)
        elif a==b:
            l.append(0)
        else:
            l.append(-1)
    if sum(l) > 0:
        return True
    else:
        return False

def profit_max(A_exclude, B_include): 
    # A, B, and idx are lists. 
    if profit_greaterthan(A_exclude, B_include):
        return A_exclude, 0
    else:
        return B_include, 1
    
def profit_addition(A, B):
    # A and B are lists. 
    return [sum(x) for x in zip(A, B)]


# Space optimized Approach (code is inspired by Prosun Kumar Sarkar)
def knapSack(W, wt, val, n): 
    # Making the dp array each with 9 category values initialized as zero.  
    dp = dp = [[[0]*9 if j==0 else [] for j in range(2)] for i in range(W+1)]

    # Taking first i elements 
    for i in range(1, n+1): 
        # Starting from back, 
        # so that we also have data of 
        # previous computation when taking i-1 items 
        for w in range(W, 0, -1): 
            if wt[i-1] <= w: 
                # Find the max value 
                dp[w][0], left_or_right = profit_max(dp[w][0], profit_addition(dp[w-wt[i-1]][0], val[i-1])) 
                if left_or_right==1: #include
                    dp[w][1] = dp[w-wt[i-1]][1].copy()
                    dp[w][1].append(i-1) 
                    
    # Returning the maximum value of knapsack 
    return dp[W]
  
  
profit = [[0,2,2,2,2,2,2,2,0], [0,100,0,0,0,0,1,0,0], [100,0,0,0,0,0,0,1,1]]
weight = [1, 1, 1]
W = 2
n = len(profit)
print(knapSack(W, weight, profit, n))



[[100, 2, 2, 2, 2, 2, 2, 3, 1], [0, 2]]


Try on our nba data df_currentweekproj = getWeekProjection_2324(current_week()). 

In [91]:
df_currentweekproj = getWeekProjection_2324(current_week())
df_currentweekproj

,slug,name,team,FGperc,FTperc,made_three_point_field_goals,points,assists,rebs,steals,blocks,turnovers
0,achiupr01,Precious Achiuwa,Team.NEW_YORK_KNICKS,0.458351,0.590238,1.994949,30.505051,4.984848,21.686869,2.146465,2.070707,4.151515
1,adebaba01,Bam Adebayo,Team.MIAMI_HEAT,0.524354,0.783249,0.097561,83.512195,15.116816,39.460847,4.718870,3.599487,10.156611
2,agbajoc01,Ochai Agbaji,Team.UTAH_JAZZ,0.375230,0.635072,4.520055,26.030429,4.132780,9.598893,2.124481,1.720609,2.849239
3,aldamsa01,Santi Aldama,Team.MEMPHIS_GRIZZLIES,0.417646,0.601087,5.272468,37.466476,5.894437,20.125535,2.356633,2.453638,3.686163
4,alexani01,Nickeil Alexander-Walker,Team.MINNESOTA_TIMBERWOLVES,0.416720,0.697632,5.432836,29.313433,9.111366,8.013777,3.200918,2.080367,4.179104
...,...,...,...,...,...,...,...,...,...,...,...,...
517,youngth01,Thaddeus Young,Team.TORONTO_RAPTORS,0.592371,0.415534,0.658228,22.472574,7.763713,13.172996,3.729958,0.886076,3.924051
518,youngtr01,Trae Young,Team.ATLANTA_HAWKS,0.432518,0.866223,11.692308,109.406593,41.938462,12.712088,4.936264,0.663736,16.813187
519,yurtsom01,Omer Yurtseven,Team.UTAH_JAZZ,0.482061,0.607295,0.474273,16.572707,2.559284,18.031320,0.859060,1.664430,3.597315
520,zelleco01,Cody Zeller,Team.NEW_ORLEANS_PELICANS,0.450095,0.640365,0.133581,13.736549,4.408163,13.922078,0.801484,0.615955,2.115028


In [92]:
df_profit = df_currentweekproj.iloc[:,3:]
df_profit.turnovers = df_profit.turnovers*-1
profit = df_profit.values.tolist()
n = len(profit)
weight = [1]*n
W = 13

values, player_idx = knapSack(W, weight, profit, n)
print(values)
print(player_idx)

df_currentweekproj.iloc[player_idx]

[6.399363485507528, 11.112601707732845, 122.10734435588951, 1343.7945290120065, 342.8351602544118, 358.8411373602369, 67.08986498009939, 45.26621978825937, -147.40702235933196]
[107, 118, 124, 130, 150, 154, 174, 235, 319, 446, 471, 495, 518]


,slug,name,team,FGperc,FTperc,made_three_point_field_goals,points,assists,rebs,steals,blocks,turnovers
107,davisan02,Anthony Davis,Team.LOS_ANGELES_LAKERS,0.555698,0.792207,1.548223,101.162437,12.720812,47.340102,4.680203,9.487310,8.680203
118,doncilu01,Luka Dončić,Team.DALLAS_MAVERICKS,0.483564,0.766711,13.972414,128.395402,35.144828,33.489655,5.250575,2.174713,15.857471
124,duranke01,Kevin Durant,Team.PHOENIX_SUNS,0.530489,0.885445,8.441926,117.694051,23.036827,26.560907,3.246459,4.679887,13.348442
130,embiijo01,Joel Embiid,Team.PHILADELPHIA_76ERS,0.532240,0.870271,4.592398,132.361730,20.010485,45.048493,4.382700,7.124509,14.411533
150,georgpa01,Paul George,Team.LOS_ANGELES_CLIPPERS,0.458699,0.902381,13.100775,94.351421,17.281654,23.540052,6.423773,1.509044,10.382429
154,gilgesh01,Shai Gilgeous-Alexander,Team.OKLAHOMA_CITY_THUNDER,0.533380,0.877839,4.875149,120.632580,24.342449,21.845422,7.995244,3.205707,9.303210
174,halibty01,Tyrese Haliburton,Team.INDIANA_PACERS,0.492497,0.865417,11.847981,83.206651,43.106888,15.942993,5.396675,2.418052,9.923990
235,jokicni01,Nikola Jokić,Team.DENVER_NUGGETS,0.582930,0.820940,4.241919,102.364964,36.175182,48.137643,5.021898,3.357664,12.608968
319,mitchdo01,Donovan Mitchell,Team.CLEVELAND_CAVALIERS,0.457575,0.870621,13.279584,108.832250,21.076723,19.079324,6.486346,1.570871,10.897269
446,tatumja01,Jayson Tatum,Team.BOSTON_CELTICS,0.474043,0.827095,12.783370,112.792123,17.934354,33.684902,4.153173,2.275711,11.326039


EXPERIMENT: "choosing the top 13 players" vs. "using knapsack algorithm".  

In [93]:
# EXPERIMENT
# try simple sorting. getting the top 13. 

df_profit = df_currentweekproj.iloc[:,3:]
df_profit.turnovers = df_profit.turnovers*-1
profit = df_profit.values.tolist()

def function_top13(P):
    top13 = []
    visited = []
    for i in range(13):
        max_p = [0]*9
        max_idx = -1
        for j, p in enumerate(P):
            if j in visited:
                continue
            if profit_greaterthan(p, max_p):
                max_p = p
                max_idx = j
        visited.append(max_idx)
        top13.append(max_idx)
    return top13

# without using knapsack algo (Top 13).
tmp1 = df_currentweekproj.iloc[function_top13(profit)]
profit_simple = [0]*7
for p in tmp1.iloc[:, 3:].values.tolist():
    profit_simple = profit_addition(profit_simple, p)

# with using knapsack algo.
tmp2 = df_currentweekproj.iloc[player_idx]
profit_knapsack = [0]*7
for p in tmp2.iloc[:, 3:].values.tolist():
    profit_knapsack = profit_addition(profit_knapsack, p)

profit_greaterthan(profit_simple, profit_knapsack)

False

We can see that the output is False, which means using the 0/1 knapsack algorithm has a greater categorical profit than just picking the top 13 players.  

Discussion:

1. Weekly Player Performance Prediction: Time Series Predictions.  
    - I anticipate to have better results than using weighted average when using Time Series predictions, such as OLS Regression, Harmonic Regression, XGBoost, Arima, Vector Autoregression...  

2. Position constraint: 1PG, 1SG, 1G, 1SF, 1PF, 1F, 2C, 2Util, 3BN  
    -  This becomes a much more complex constraint optimization problem such as Mulitple Knapsack Assignment Problem (MKAP)

3. Snake Draft constraint: 
    - Simulate the snake draft: for simplicity, round1 only drafts within 1~12 picks, round2: 12~23, round3: 24~35,...   
    - Build a live draft version.  

4. Consider Accidents: Player Injuries, Game Time Decisions, Suspensions ...